In [ ]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


In [5]:
import os


EPOCHS = 100
BATCH_SIZE = 8
ARCH = "flor"

IMG_SIZE = (128,32, 1)
DATA_ROOT_PATH = "..\\data"
IMAGES_PATH = os.path.join(DATA_ROOT_PATH, "images", "data")
IMAGES_PATH

'..\\data\\images\\data'

In [6]:
vocab = None

with open(os.path.join(DATA_ROOT_PATH, "hindi_vocab.txt"),encoding="utf-8") as f:
  vocab = f.readlines()

idx_to_vocab = {i:value.strip() for i, value in enumerate(vocab)}
vocab_to_idx = {value:key for key, value in idx_to_vocab.items()}

In [7]:
train_data = None

with open(os.path.join(DATA_ROOT_PATH, "new_train.txt"), encoding="utf-8") as f:
  train_data = f.readlines()

In [8]:
valid_data = None

with open(os.path.join(DATA_ROOT_PATH, "new_val.txt"), encoding="utf-8") as f:
  valid_data = f.readlines()

In [9]:
test_data = None

with open(os.path.join(DATA_ROOT_PATH, "new_val.txt"), encoding="utf-8") as f:
  test_data = f.readlines()

In [10]:
charl = None

with open( "charList.txt", encoding="utf-8") as f:
  charl = f.readlines()

In [11]:
charl

[' -ँंःअआइईउऊऋऌऍऎएऐऑऒओऔकखगघङचछजझञटठडढणतथदधनऩपफबभमयरऱलळऴवशषसह़ऽािीुूृॄॅेैॉॊोौ्ॐ॒॑॓॔क़ख़ग़ज़ड़ढ़फ़य़ॠॢ।॥०१२३४५६७८९॰ॱॲॻॼॽॾ']

In [12]:
charl=charl[0]

In [13]:
train_data=train_data[:-3]

In [14]:
len(train_data)

69800

# train data labels

# train data generation

In [15]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from PIL import Image, ImageOps
from data import preproc as pp
from data import preproc1 as pp1

In [16]:

##label generator
def label_g(l):
    label=idx_to_vocab[l]
    z=[]
    for j in range(50):
        z.append(0)
    for k in range(len(label)):
        for r in range(len(charl)):
            if label[k]==charl[r]:
                z[k]=r
    return z   

In [17]:

##label generator
def label_g1(l):
    label=idx_to_vocab[l]
    z=[]
    for k in range(len(label)):
        for r in range(len(charl)):
            if label[k]==charl[r]:
                z.append(r)
    return z   

In [18]:
s=np.zeros((8, 27))
s

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [19]:
len(train_data)

69800

In [20]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from PIL import Image, ImageOps
from data import preproc as pp
import cv2

class DataGen(tf.keras.utils.Sequence):
    def __init__(self, data, img_size, batch_size, mode="TRAIN"):
        self.data = data
        self.img_size = img_size
        self.batch_size = batch_size
        self.mode = mode
        
    def __getitem__(self, i):
        #print(i)
        start = i * self.batch_size
        end = (i+1) * self.batch_size
        batch_images = np.zeros((self.batch_size, self.img_size[0], self.img_size[1], 1))
        batch_labels = np.zeros((self.batch_size, 50))
        #batch_labels =[[],[],[],[],[],[],[],[]]
        #print(start)
        for ii, df_index in enumerate(range(start, end)):
            #print(ii)
            #print(df_index)
            curr_data = self.data[df_index].split()
            curr_img_path = curr_data[0]
            #curr_label = idx_to_vocab[int(curr_data[1])]
            curr_label = label_g(int(curr_data[1]))

            curr_img_path = "/".join(curr_img_path.split("/")[2:])
            curr_img_path = os.path.join(IMAGES_PATH, curr_img_path)

            
            curr_img = pp.preprocess(curr_img_path, self.img_size)
            curr_img=cv2.adaptiveThreshold(curr_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 199, 5)
            #curr_img = tf.image.resize(curr_img, (self.img_size[0], self.img_size[1]), method="nearest")
            #curr_img = curr_img.numpy().reshape((self.img_size[0], self.img_size[1],1))
            batch_images[ii, :, :,0] = curr_img 
            batch_labels[ii,:] = curr_label
                    
        if self.mode == "TRAIN":
          return batch_images, batch_labels
        else:
          return batch_images
        
    def __len__(self):
      return len(self.data) // self.batch_size

In [21]:
train_datagen = DataGen(train_data, IMG_SIZE, 8)

In [22]:
train_datagen[4364][1]

array([[50., 61., 47.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [46., 60., 31., 62.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [46., 63., 57., 50., 74., 50., 72.,  3.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [27., 72., 31., 68., 42., 63., 48.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
     

In [ ]:
test_datagen = DataGen(test_data, IMG_SIZE, 10000)

# model


In [23]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import Model
from tensorflow.keras.constraints import MaxNorm

from network.layers import FullGatedConv2D, GatedConv2D, OctConv2D
from tensorflow.keras.layers import Conv2D, Bidirectional, LSTM, GRU, Dense
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, PReLU
from tensorflow.keras.layers import Input, Add, Activation, Lambda, MaxPooling2D, Reshape

In [24]:
from network.model import HTRModel

# create and compile HTRModel
model = HTRModel(architecture=ARCH,
                 input_size=IMG_SIZE,
                 vocab_size=110,
                 beam_width=10,
                 stop_tolerance=20,
                 reduce_tolerance=15,
                 reduce_factor=0.1)

model.compile(learning_rate=0.001)

ValueError: Exception encountered when calling layer "max_pooling2d" (type MaxPooling2D).

Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling2d/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 1, 2, 1], padding="VALID", strides=[1, 1, 2, 1]](Placeholder)' with input shapes: [?,16,1,64].

Call arguments received by layer "max_pooling2d" (type MaxPooling2D):
  • inputs=tf.Tensor(shape=(None, 16, 1, 64), dtype=float32)

In [20]:
model.fit(x=train_datagen,
          
              epochs=EPOCHS,
              verbose=1)

Epoch 1/100
4364/8725 [==============>...............] - ETA: 32:46 - loss: 18.7849

InvalidArgumentError: Graph execution error:

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'
Traceback (most recent call last):

  File "C:\Users\pavan\anaconda3\envs\pavan\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "C:\Users\pavan\anaconda3\envs\pavan\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\pavan\anaconda3\envs\pavan\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1039, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\pavan\anaconda3\envs\pavan\lib\site-packages\keras\engine\data_adapter.py", line 901, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\pavan\anaconda3\envs\pavan\lib\site-packages\keras\engine\data_adapter.py", line 1048, in generator_fn
    yield x[i]

  File "C:\Users\pavan\A

In [22]:
train_data[4364*8:4365*8]

['HindiSeg/train/1/86/22.jpg 9684\n',
 'HindiSeg/train/1/86/23.jpg 9036\n',
 'HindiSeg/train/1/86/24.jpg 9192\n',
 'HindiSeg/train/1/86/25.jpg 6451\n',
 'HindiSeg/train/1/86/26.jpg 8202\n',
 'HindiSeg/train/1/86/27.jpg 3919\n',
 'HindiSeg/train/1/86/28.jpg 2962\n',
 'HindiSeg/train/1/86/29.jpg 4027\n']

In [23]:
4364*8

34912

In [40]:
curr_data = train_data[34928].split()
curr_img_path = curr_data[0]


curr_img_path = "/".join(curr_img_path.split("/")[2:])
curr_img_path = os.path.join(IMAGES_PATH, curr_img_path)
curr_img = pp.preprocess(curr_img_path, IMG_SIZE)

In [ ]:
curr_img1.shape

In [ ]:
curr_img1 = np.expand_dims(curr_img, 0)

In [ ]:
model.predict(curr_img1)

In [ ]:
print(charl[56])
print(charl[61])
print(charl[62])
print(charl[21])
print(charl[3])
print(charl[23])

In [ ]:
plt.imshow(curr_img)

In [ ]:
print(charl[57])
print(charl[60])
print(charl[3])
print(charl[23])
print(charl[21])
print(charl[60])
print(charl[3])
print(charl[23])

In [ ]:
xt=test_datagen[0][0]
yt=test_datagen[0][1]

In [ ]:
predicts, _ = model.predict(x=xt,
                            ctc_decode=True,
                            verbose=1)

In [ ]:
len(predicts[0])

In [ ]:
predicts[1][0]

In [ ]:
def convert(predicts):
    a=[]
    b=[]
    for i in range(len(predicts)):
        b=[]
        for j in range(len(predicts[i][0])):
            b.append(charl[predicts[i][0][j]])
        a.append(b)
    return a

In [ ]:
def convert1(predicts):
    a=[]
    b=[]
    for i in range(len(predicts)):
        b=[]
        for j in range(len(predicts[i])):
            b.append(charl[int(predicts[i][j])])
        a.append(b)
    return a

In [ ]:
def trim(yt):
    yt1=[]
    for i in range(len(yt)):
        k=0
        for j in range(26,0,-1):
            if yt[i][j]!=0:
                yt1.append(yt[i][0:j+1])
                k=1
                break
        if k==0:
            yt1.append([0])
    return yt1

In [ ]:
yt1=trim(yt)

In [ ]:
len(yt1[0])

In [ ]:
len(yt[0])

In [ ]:
yt1[0][1]

In [ ]:
predicts1=convert(predicts)

In [ ]:
gt=convert1(yt1)

In [ ]:
from data import evaluation

In [ ]:

import string
import unicodedata
import editdistance
import numpy as np


def ocr_metrics(predicts, ground_truth, norm_accentuation=False, norm_punctuation=False):
    """Calculate Character Error Rate (CER), Word Error Rate (WER) and Sequence Error Rate (SER)"""

    if len(predicts) == 0 or len(ground_truth) == 0:
        return (1, 1, 1)

    cer, wer, ser = [], [], []

    for (pd, gt) in zip(predicts, ground_truth):
        '''pd, gt = pd.lower(), gt.lower()

        if norm_accentuation:
            pd = unicodedata.normalize("NFKD", pd).encode("ASCII", "ignore").decode("ASCII")
            gt = unicodedata.normalize("NFKD", gt).encode("ASCII", "ignore").decode("ASCII")

        if norm_punctuation:
            pd = pd.translate(str.maketrans("", "", string.punctuation))
            gt = gt.translate(str.maketrans("", "", string.punctuation))
	'''
        pd_cer, gt_cer = list(pd), list(gt)
        dist = editdistance.eval(pd_cer, gt_cer)
        cer.append(dist / (max(len(pd_cer), len(gt_cer))))
'''
        pd_wer, gt_wer = pd, gt
        dist = editdistance.eval(pd_wer, gt_wer)
        wer.append(dist / (max(len(pd_wer), len(gt_wer))))
        
        pd_ser, gt_ser = [pd], [gt]
        dist = editdistance.eval(pd_ser, gt_ser)
        ser.append(dist / (max(len(pd_ser), len(gt_ser))))
        '''
    metrics = [cer, wer]
    metrics = np.mean(metrics, axis=1)

    return metrics

In [ ]:
evaluate = ocr_metrics(predicts=predicts1,
                                  ground_truth=gt,)
 
print("Calculate Character Error Rate {} ".format(evaluate[0],))

In [ ]:
predicts1[0]

# charlist generation
